In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [10]:
def test_main():
    ''' TEST ALL RAW METHODS '''
    
    # Load train and test data
    DATA_TRAIN_PATH = "../data/train.csv"
    y, tx, ids = load_csv_data(DATA_TRAIN_PATH)
    
    print('Data loaded')
    
    methods = ['least_square_GD', 
               'least_square_SGD', 
               'least_squares', 
               'ridge_regression', 
               'logistic_regression', 
               'reg_logistic_regression']
    
    scores = []
    
    scores.append(test_methods(y,tx,methods[0]))
    scores.append(test_methods(y,tx,methods[1]))
    scores.append(test_methods(y,tx,methods[2]))
    scores.append(test_methods(y,tx,methods[3]))
    scores.append(test_methods(y,tx,methods[4]))
    scores.append(test_methods(y,tx,methods[5]))
    
    index = np.argmax(scores)   
    print('Best method is ', methods[index], ': score = ', scores[index])
    
test_main()

Data loaded


In [22]:
def main():
    '''OPTIMAL METHOD COMPUTED
    '''
    # Load train and test data
    DATA_TRAIN_PATH = "../data/train.csv"
    y_train, tx_train, ids = load_csv_data(DATA_TRAIN_PATH)
    
    # Split in jet num
    jet_nums = range(0,4,1)
    
    ids_jets = []
    tx_jets = []
    y_jets = []
    
    for i in jet_nums:
        
        # find data with jet num i
        data_jet = np.where(tx_train[:,22]==i)
        ids_jet = ids[data_jet]
    
        
        # creat sub label and sub feature data
        tx_jet = tx_train[data_jet]
        y_jet = y_train[data_jet]
        
        # process data
        tx_jet = process_data(tx_jet)      
        
        # store
        tx_jets.append(tx_jet)
        y_jets.append(y_jet)
        
        print('tx jet: ',np.shape(tx_jet))
        print('ids jet: ', np.shape(ids_jet))
    
    print('Data loaded')
    
    
    ws = []
    # optimal hyperparameters found by grid search for each jet num
    optimal_degree = [6, 7, 6, 6]
    optimal_lambda = [1.0811807510766078e-10, 2.20673406908459e-06, 8.227241341700474e-06, 3.1011689265747817e-07]
    
    print('Hyperparameters optimized')
    for i in jet_nums:
 
        # if grid_search:
        #optimal_degree, optimal_lambda = find_optimal_hyperparameters(y_jets[i+1], tx_jets[i+1])
    
   
        # Build poly and add cross-terms and logs

        print('Building training polynomial array ongoing...')
        final_tx_jet = build_poly(tx_jets[i], optimal_degree[i])
        final_tx_jet = np.c_[final_tx_jet, build_cross_terms(tx_jets[i])]
        print('shape of final_tx_train: ',np.shape(final_tx_jet))

        print('Ridge regression ongoing')
        w, loss = ridge_regression(y_jets[i], final_tx_jet, optimal_lambda[i])
        print('shape of weights: ',np.shape(w))

        ws.append(w)

    return ws

w = main()

tx jet:  (99913, 18)
ids jet:  (99913,)
tx jet:  (77544, 22)
ids jet:  (77544,)
tx jet:  (50379, 29)
ids jet:  (50379,)
tx jet:  (22164, 29)
ids jet:  (22164,)
Data loaded
Hyperparameters optimized
Building training polynomial array ongoing...
shape of final_tx_train:  (99913, 262)
Ridge regression ongoing
shape of weights:  (262,)
Building training polynomial array ongoing...
shape of final_tx_train:  (77544, 386)
Ridge regression ongoing
shape of weights:  (386,)
Building training polynomial array ongoing...
shape of final_tx_train:  (50379, 581)
Ridge regression ongoing
shape of weights:  (581,)
Building training polynomial array ongoing...
shape of final_tx_train:  (22164, 581)
Ridge regression ongoing
shape of weights:  (581,)


In [ ]:
def jet_num_main(w):
    
    DATA_TEST_PATH = "../data/test.csv"
    _, tx_test, ids_test = load_csv_data(DATA_TEST_PATH)

    print('Data loaded')
    
    final_tx_test= []
    
    ids_jets = []
    tx_jets = []
    
    y_pred = np.zeros((tx_test.shape[0],1))
    print('y_pred shape: ', np.shape(y_pred))
    jet_num = range(0,4,1)
    
    for i in jet_num:
        
        # create sub data jet
        data_jet = np.where(tx_test[:,22]==i)
        ids_jet = ids_test[data_jet]
        tx_jet = tx_test[data_jet]
        
        # process data
        tx_jet = process_data(tx_jet) 
        
        # store data
        tx_jets.append(tx_jet)
        ids_jets.append(ids_jet)
        
        print('tx jet: ',np.shape(tx_jet))
        print('ids jet: ', np.shape(ids_jet))
            
    # optimal hyperparameters found by grid search for each jet num
    optimal_degree = [6, 7, 6, 6]
    optimal_lambda = [1.0811807510766078e-10, 2.20673406908459e-06, 8.227241341700474e-06, 3.1011689265747817e-07]
     
    for i in jet_num:
        
        final_tx_jet = build_poly(tx_jets[i], optimal_degree[i])
        print('poly built')
        print(np.shape(final_tx_jet))
        final_tx_jet = np.c_[final_tx_jet, build_cross_terms(tx_jets[i])]
        print('cross built')
        print('shape of final_tx_test: ', np.shape(final_tx_jet))

        y_pred_jet = predict_labels(w[i], final_tx_jet)
        print('shape y_pred_jet: ',np.shape(y_pred_jet))
        
        for j in range(len(y_pred_jet)):
            
            y_pred[ids_jets[i][j]-350000,0] = y_pred_jet[j]
        
    y_pred.tolist()
    
    print(np.shape(y_pred))
    
    return y_pred, ids_test

y_pred, ids_test = jet_num_main(w)

Data loaded
y_pred shape:  (568238, 1)
tx jet:  (227458, 18)
ids jet:  (227458,)
tx jet:  (175338, 22)
ids jet:  (175338,)
tx jet:  (114648, 29)
ids jet:  (114648,)
tx jet:  (50794, 29)
ids jet:  (50794,)
poly built
(227458, 109)
cross built
shape of final_tx_test:  (227458, 262)
shape y_pred_jet:  (227458,)
poly built
(175338, 155)
cross built
shape of final_tx_test:  (175338, 386)
shape y_pred_jet:  (175338,)
poly built
(114648, 175)


In [21]:
OUTPUT_PATH = "../data/submission017.csv"
#y_pred = predict_labels(weights, tX_test)
#y_pred = predict_labels(w, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)